In [2]:
# This is its own cell because it takes a while to load this thing
from gensim import models

# takes a lilbit
# model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  

In [3]:
# imports and permanent stuff

import requests
import json
import pickle
import os
import numpy as np
import time
import shutil

In [382]:
## class Loader

class Loader:
    def __init__(self, list_of_ids = [], list_of_times = []):
        self.list_of_ids = list_of_ids
        self.list_of_times = list_of_times
    
    filedir = ""
    #list_of_ids = []
    #list_of_times = []
    list_of_types = ['text','log','contact','calender','file','tweets', 'Instagram', 'Instagram media', 'phq']
    
    def deletdatboi(self, nombre):
        shutil.rmtree('./datafor' + list_of_scrape_times[nombre])
        
    def lids(self):
        return self.list_of_ids
    
    def lits(self):
        return self.list_of_times
    
    def downloadAndLabel(self):
        
        r = requests.get('http://depressionmqp.wpi.edu:8080/getids')
        list_of_idtime = r.json()
        

        for i in range(0,len(list_of_idtime)):
            self.list_of_ids.append( list_of_idtime[i]['id'].encode('ascii','ignore') )

        for i in range(0,len(list_of_idtime)):
            self.list_of_times.append( list_of_idtime[i]['date'] )

        number_cols = len(self.list_of_types)
        number_rows = len(self.list_of_ids)
        
        # an artifact of the past
        # list_of_jsons = [[0] * number_cols for i in range(number_rows)]

        
        ### create directory for this particular scrape/pull
        timenow  = str(int(time.time())) # for temporal congruency
        list_of_scrape_times.append(timenow)
        os.mkdir('./datafor' + timenow[-3:])
        Loader.filedir = './datafor' + timenow[-3:]


        for i in range(0,number_rows):
            for j in range(0,number_cols):
                temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(int(self.list_of_ids[i])) + '&type=' + self.list_of_types[j])

                pickle.dump(temp, open( Loader.filedir + "/DP" + str(int(self.list_of_ids[i])) +  self.list_of_types[j] + ".p", "wb" ))

                # loads it into memory. i will not use this for now
                # for sake of architectural sanity
                # list_of_jsons[i][j] = temp


In [593]:
## Featurizer featurizes methods that convert json objects of the appropriate type into features

class Featurizer:
    def __init__(self):
        pass
        #self.name = name

    # takes in element of list_of_jsons and scrape date, spits out text frequency
    def dailyTextFreq(self, texts, scrapedate):
        
        a = texts.json()

        seconds_intwo_weeks = 1209600;
        
        # calculate unix time for two weeks prior
        two_weeks_prior = scrapedate - seconds_intwo_weeks

        text_date = 0
        saved_index = 0

        # find the index of the oldest message that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            text_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < text_date):
                saved_index = len(a) - (i+1)
                break

        # divide by 14 to get daily freq
        return float(saved_index+1)/14
    

    # takes in element of list_of_jsons(resp object) and scrape date, spits out call frequency 
    def dailyCallFreq(self, calls, scrapedate):

        a = calls.json()

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        call_date = 0
        saved_index = 0

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            call_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < call_date):
                saved_index = len(a) - (i+1)
                break

        return float(saved_index+1)/14
    
    
    #input is tweets, return master vector
    def embeddingToMastersum(self, tweets):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,10):
            try:
                masterSum += self.tweetToEmbedding(tweets.json()[i])
            except:
                pass
        
        return masterSum

    
    # input is tweet, returns vector embedding of entire tweet.
    # eg: responseobject.json()[0]
    def tweetToEmbedding(self, tweet):

        q = json.loads(tweet)['text'].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    #input is tweets, return follocount
    def followerCount(self, tweets):

        followerCount = 0

        # get
        try:
            followerCount = json.loads(tweets.json()[0])['user']['followers_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followerCount

    #input is tweets, return friends (who user follows)
    def followingCount(self, tweets):

        followingCount = 0

        try:
            followingCount = json.loads(tweets.json()[0])['user']['friends_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followingCount

    #input is tweets, return avg likes per post for the last 2 weeks
    def twitterLikeFreq(self, tweets, scrapedate):

        a = tweets.json()

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_likes = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets.json()[i])['favorite_count']


        return sum_of_likes/14

    #input is tweets, return avg retweets per post for the last 2 weeks
    def twitterRetweetFreq(self, tweets, scrapedate):

        a = tweets.json()

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_retweets = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets.json()[i])['retweet_count']


        return sum_of_retweets/14
    
    # input is contacts, returns number of contacts
    def numOfContacts(self, contacts):
        return len(contacts.json())
    
    #input is instagram, return two features: follows count, followed by count  
    def instagramThings(self, instagram):
        
        followsFollowed = np.zeros((2,))
        
        
        try:
            if(type(json.loads(instagram.json()[0])) == str):
                print("DAMN YOU DAMON!!(ig)")
            else: # its a dictionary like it's supposed to be
                followsFollowed[0] = json.loads(instagram.json()[0])['data']['counts']['follows']
                followsFollowed[1] = json.loads(instagram.json()[0])['data']['counts']['followed_by']
        except IndexError:
            pass
        
        return followsFollowed
    
    # takes in scrape date, spits out filter usage frequency 
    def instagramFilterFreq(self, instagramMedia, scrapedate):
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia.json()

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = instagramMedia.json()[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break

                    
        negative_count = 0

        for i in range(0,saved_index):
            if(json.loads(a[i])['filter'] == Normal):
                negative_count += 1

        # filter usage freq = posts with filters count / total post count 
        return ((saved_index+1)-negative_count)/(saved_index+1)
    
    
    def instagramFilterVector(self, instagramMedia, scrapedate):
        
        filters = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
        filtervec = np.zeros((8,))
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia.json()

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = instagramMedia.json()[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break

                    
        for i in range(0,saved_index):
            filt = json.loads(a[i])['filter']
            if(filt in filters):
                filters[filt] += 1
            else:
                pass
            
        for i in range(0,8):
            filtervec[i] = filters[list(filters)[i]]

        # percentage of how much the filters are used as a normalized vector
        return filtervec/(saved_index+1)
    
    def instagramLikeComFreq(self, instagramMedia, scrapedate):
        
        counts = np.zeros((2,))
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia.json()

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = instagramMedia.json()[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break
                
        for i in range(0,saved_index):
            counts[0] += json.loads(instagramMedia.json()[i])['likes']['count']
            counts[1] += json.loads(instagramMedia.json()[i])['comments']['count']
        
            
        # likes per post for past 2 weeks
        if (not(counts[0] != 0 and counts[1] != 0)):
            return np.zeros((2,))
        else:
            return counts/saved_index

    # IG post frequency for the past 2 weeks
    def instagramPostFreq(self, instagramMedia, scrapedate):
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia.json()

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = instagramMedia.json()[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break
                
        return (saved_index+1)/14
        
        


In [582]:
class Generator:

    def __init__(self, filedir, list_of_ids, list_of_times):
        self.filedir = filedir
        self.list_of_ids = list_of_ids
        self.f = Featurizer()
        self.num_of_people = len(list_of_ids)
        self.list_of_times = list_of_times
    
    featureMatrix = None
    
    def generateMatrix(self):

        featureVectorCF = np.zeros((self.num_of_people,1))
        featureVectorTF = np.zeros((self.num_of_people,1))
        featureVectorTW = np.zeros((self.num_of_people,300))
        featureVectorFC = np.zeros((self.num_of_people,1))
        featureVectorFC2 = np.zeros((self.num_of_people,1))
        featureVectorTWL = np.zeros((self.num_of_people,1))
        featureVectorTWRT = np.zeros((self.num_of_people,1))
        featureVectorNC = np.zeros((self.num_of_people,1))
        featureVectorIG1 = np.zeros((self.num_of_people,2))
        featureVectorIG2 = np.zeros((self.num_of_people,1))
        featureVectorIGFV = np.zeros((self.num_of_people,8))
        featureVectorIGLC = np.zeros((self.num_of_people,2))
        featureVectorIG3 = np.zeros((self.num_of_people,1))
        
        for i in range(0, self.num_of_people):

            #list_of_jsons[i][1]

            a1 = pickle.load( open( self.filedir + "/DP" + str(int(self.list_of_ids[i])) +  "log" + ".p", "rb" )) 
            a2 = pickle.load( open( self.filedir + "/DP" + str(int(self.list_of_ids[i])) +  "text" + ".p", "rb" )) 
            a3 = pickle.load( open( self.filedir + "/DP" + str(int(self.list_of_ids[i])) +  "tweets" + ".p", "rb" )) 
            a4 = pickle.load( open( self.filedir + "/DP" + str(int(self.list_of_ids[i])) +  "contact" + ".p", "rb" )) 
            a5 = pickle.load( open( self.filedir + "/DP" + str(int(self.list_of_ids[i])) +  "Instagram" + ".p", "rb" )) 
            a6 = pickle.load( open( self.filedir + "/DP" + str(int(self.list_of_ids[i])) +  "Instagram media" + ".p", "rb" )) 
            
            # print(self.list_of_ids[i])
            # print(i)
            
            featureVectorCF[i] = self.f.dailyCallFreq(a1, self.list_of_times[i])
            featureVectorTF[i] = self.f.dailyTextFreq(a2, self.list_of_times[i])
            featureVectorTW[i] = self.f.embeddingToMastersum(a3)
            featureVectorFC[i] = self.f.followerCount(a3)
            featureVectorFC2[i] = self.f.followingCount(a3)
            featureVectorTWL[i] = self.f.twitterLikeFreq(a3, self.list_of_times[i])
            featureVectorTWRT[i] = self.f.twitterRetweetFreq(a3, self.list_of_times[i])
            featureVectorNC[i] = self.f.numOfContacts(a4)
            featureVectorIG1[i] = self.f.instagramThings(a5)
            featureVectorIG2[i] = self.f.instagramFilterFreq(a6, self.list_of_times[i])
            featureVectorIGFV[i] = self.f.instagramFilterVector(a6, self.list_of_times[i])
            featureVectorIGLC[i] = self.f.instagramLikeComFreq(a6, self.list_of_times[i])
            instagramPostFreq
            

            Generator.featureMatrix = np.hstack((featureVectorCF,featureVectorTF,
featureVectorTW, featureVectorFC, featureVectorTWL, featureVectorTWRT,
featureVectorNC,featureVectorIG1, featureVectorIG2, featureVectorIGFV,
featureVectorIGLC))


In [590]:
# Everything I coded in one cell!

# l = Loader()
# l.downloadAndLabel()

g = Generator(l.filedir, l.lids(), l.lits())
g.generateMatrix()
g.featureMatrix.shape

/home/vape/.local/lib/python3.5/site-packages/ipykernel_launcher.py:319: RuntimeWarning: invalid value encountered in true_divide


DAMN YOU DAMON!!(ig)
DAMN YOU DAMON!!(ig)
DAMN YOU DAMON!!(ig)
DAMN YOU DAMON!!(ig)


(140, 319)

In [567]:
## WASTELAND OF PLAY CELLS BELOW


filedir = './datafor' + '288'

## access example

# instagram = pickle.load( open( filedir + "/DP" + str(int(l.lids()[4])) +  "tweets" + ".p", "rb" )) 

a3 = pickle.load( open( filedir + "/DP" + '1995377' +  "Instagram media" + ".p", "rb" )) 
# a3 = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 
#instagram = pickle.load( open( filedir + "/DP" + '11852603' +  "Instagram" + ".p", "rb" )) 



# json.loads(a3.json()[0])

a3.json()





[]

In [525]:
kelsey = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}

# for fil in kelsey:
#     print(kelsey[fil])

'Valencia' in kelsey

True

In [552]:
instagramMedia = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 
filters = {'Lark':0,'Slumber':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
filtervec = np.ones((8,))

for i in range(0,3):
    filt = json.loads(a[i])['filter']
    if(filt in filters):
        filters[filt] += 1
    else:
        pass
    
for i in range(0,8):
    filtervec[i] = filters[list(filters)[i]]
    
filtervec

array([ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.])

In [591]:
## TWITTER 70522438
## IG   19671950


# instagramMedia = pickle.load( open( filedir + "/DP" + '82562975' +  "Instagram media" + ".p", "rb" )) 
instagramMedia = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 

len(instagramMedia.json())


2

In [520]:
filtervec = np.ones((8,))
filtervec/4

array([ 0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25])

In [143]:
w = embeddingToMastersum(a3)
w.reshape((-1, 1)).T.shape


(1, 300)

In [156]:
timenow  = str(int(time.time())) # for temporal congruency
timenow1 = timenow
timenow1

'1511765588'

In [159]:
timenow[-3:]

'588'